In [1]:
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle 
import pandas as pd
from scipy.stats import randint
import numpy as np
import string
import time
import base64
import re
import sys

import contractions

# librairie BeautifulSoup
from bs4 import BeautifulSoup

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud

## detection de language
import langdetect 

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import RegexpParser
# il est possible de charger l'ensemble des librairies en une seule fois 
# décocher le commentaire de la ligne ci-dessous
#nltk.download('all') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords

import spacy
from spacy.tokens import Span
# il faut sélectionner pour quelle langue les traitements vont être faits.




[nltk_data] Downloading package punkt to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2023-03-13 11:32:18.520848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-crit

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [25]:
nltk.download("stopwords")
from nltk.corpus import stopwords
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import pandas as pd
df = pd.read_csv('data/HAI817_Projet_train.csv')
df.head()

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,mixture
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,mixture
3,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false


# Tokenization
### Création de liste des mots qui apparaissent dans le texte et dans le titre de chaque document 

In [45]:
def get_tokens(dataframe):    
    full_texts = []
    for i in range(len(dataframe)):
        full_texts.append((str(dataframe.iloc[i].title) + " " + str(dataframe.iloc[i].text)))
    tokens = [t.split() for t in full_texts]
    tokens = [[t.lower() for t in lt] for lt in tokens]
    return tokens


In [61]:
def elim_stopwords(words, stopwords):
    return [word for word in words if word not in stopwords]

    

In [31]:
def stemming(dataframe):
    """ """
    ps=nltk.stem.porter.PorterStemmer()
    dataframe["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])
    return dataframe


In [93]:
def prepare_data(dataframe, classes):
    dataframe["tokens"] = get_tokens(dataframe)
    
    swords = set(stopwords.words("english"))
    dataframe["tokens"] = dataframe["tokens"].transform(lambda t: [word for word in t if word not in swords])
    
    ps=nltk.stem.porter.PorterStemmer()
    dataframe["tokens"] = dataframe["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])

    dataframe = dataframe[dataframe["our rating"].isin(classes)]
    dataframe = dataframe.reset_index(drop=True)
        
    return dataframe

In [94]:
df_train = prepare_data(df, ["true", "false"])


In [95]:
(display(df_train.head(25)))

,public_id,text,title,our rating,tokens
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,"[fine, $1,500, passeng, use, mobil, phone,, st..."
1,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false,"[obama’, daughter, caught, camera, burn, us, f..."
2,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false,"[leak, visitor, log, reveal, schiff’, 78, visi..."
3,61bd9a69,"November 23, 2019 The U.S. Food and Drug Admi...",FDA Shocking Study: Cells Used In Vaccines Con...,false,"[fda, shock, study:, cell, use, vaccin, contam..."
4,bb1999cc,"Trump confirms this was a bombing, not an acci...","Israel Hits Beirut with Nuclear Missile, Trump...",false,"[israel, hit, beirut, nuclear, missile,, trump..."
5,c1dc1ac6,In a show of anti-American sentiment that will...,Obama’s Daughters Caught on Camera Burning US ...,false,"[obama’, daughter, caught, camera, burn, us, f..."
6,f2182a54,"FEMA camps, portable human cages, it's all rea...",Fields of human cages discovered in Caruthers ...,false,"[field, human, cage, discov, caruth, californi..."
7,2ec747ae,Submitted by Great Game India Last year a mys...,Did China Steal Coronavirus From Canada And We...,false,"[china, steal, coronaviru, canada, weapon, it?..."
8,c69de08e,The proportion of people living in poverty who...,Number of people in poverty in working familie...,false,"[number, peopl, poverti, work, famili, hit, re..."
9,392886ea,"“I love what he says, I have to admit.” That w...",Kenny Rogers’ Wife Donates Half His Estate To ...,false,"[kenni, rogers’, wife, donat, half, estat, tru..."


In [96]:
from sklearn.feature_extraction.text import CountVectorizer

def create_bag(dataframe):
    corpus = dataframe["tokens"].transform(lambda s: " ".join(s)).tolist()
    vectorizer = CountVectorizer()
    # creation du vocabulaire
    vector = vectorizer.fit_transform(corpus)
    df = pd.DataFrame(data=vectorizer.transform(corpus).toarray(), columns=vectorizer.get_feature_names_out())
    df.insert(0, "our_rating", dataframe["our rating"])
    return (vectorizer, df)
    


In [97]:
bag, bag_train = create_bag(df_train)


In [99]:
bag_train.head()

,our_rating,00,000,0008,000m,000x,001,002,003,005,...,zucker,zuckerburg,zundel,zurich,zwar,zxc21,zzt2briay,ärzteblatt,único,μm
0,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [105]:
X = bag_train.values[:, 1:]
y = bag_train.values[:, 0]

models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC(gamma='auto')))
models.append(('LSVC', LinearSVC()))
models.append(('RFC', RandomForestClassifier()))

seed = 7
results = []
names = []
scoring='accuracy'
for name,model in models:
    kfold = KFold(n_splits=10, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) 

/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

LR: 0.831435 (0.040826)
KNN: 0.762999 (0.051546)
NB: 0.783268 (0.048845)
SVC: 0.749075 (0.059213)


/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244

KeyboardInterrupt: 

In [108]:
clf = LogisticRegression()
clf.fit(X, y)
res = clf.score(X, y)
print(res)

/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9987325728770595
